# Osprey usage

#### Osprey is a WPS server that runs stream flow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from datetime import datetime

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'http://localhost:5002/wps'
osprey = WPSClient(url)

#### The list of available processes can be diplayed using the ? command (ex. bird?).

In [2]:
# NBVAL_IGNORE_OUTPUT
osprey?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7f83783caba8>
File:            ~/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Data Analysis.

Processes
---------

convert
    A simple conversion utility to provide users with the ability to convert old routing model setups into RVIC parameters.

convolution
    Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

parameters
    Develop impulse response functions using inputs from a configuration file or dictionary

full_rvic
    Run full RVIC process combining Parameters and Convolution modules
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS p

Furthermore, help for individual processes can be diplayed using the same command (ex. bird.process?).

In [3]:
# NBVAL_IGNORE_OUTPUT
osprey.convert?

Signature: osprey.convert(convert_config, loglevel='INFO')
Docstring:
A simple conversion utility to provide users with the ability to convert old routing model setups into RVIC parameters.

Parameters
----------
convert_config : string
    Path to input configuration file or input dictionary
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-0>
Type:      method


In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

Signature: osprey.parameters(params_config, np=1, version=True, loglevel='INFO')
Docstring:
Develop impulse response functions using inputs from a configuration file or dictionary

Parameters
----------
params_config : string
    Path to input configuration file or input dictionary
np : integer
    Number of processors used to run job
version : boolean
    Return RVIC version string
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-2>
Type:      method


In [5]:
# NBVAL_IGNORE_OUTPUT
osprey.convolution?

Signature: osprey.convolution(convolve_config, loglevel='INFO')
Docstring:
Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

Parameters
----------
convolve_config : string
    Path to input configuration file or input dictionary
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-1>
Type:      method


In [6]:
# NBVAL_IGNORE_OUTPUT
osprey.full_rvic?

Signature:
osprey.full_rvic(
    params_config,
    convolve_config,
    version=True,
    np=1,
    loglevel='INFO',
)
Docstring:
Run full RVIC process combining Parameters and Convolution modules

Parameters
----------
params_config : string
    Path to parameters module's input configuration file or input dictionary
convolve_config : string
    Path to convolution module's input configuration file or input dictionary
version : boolean
    Return RVIC version string
np : integer
    Number of processors used to run job
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-3>
Type:      method


#### Now that we know which process we wish to run, we can use the docstring to ensure we provide the appropriate parameters.

In [7]:
# run convert
convert_output = osprey.convert(
    convert_config = "tests/data/configs/convert_local.cfg"
)

In [8]:
# run parameters
params_output = osprey.parameters(
    params_config = "tests/data/configs/parameters_local.cfg"
)

In [9]:
# run convolution
convolve_output = osprey.convolution(
    convolve_config = "tests/data/configs/convolve_opendap.cfg"
)

In [10]:
# run full_rvic
full_rvic_output = osprey.full_rvic(
    params_config = "tests/data/configs/parameters_local.cfg",
    convolve_config = "tests/data/configs/convolve_opendap.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [11]:
def make_tmp_copy(output):
    output_content = requests.get(output.get()[0]).content

    tmp_copy = NamedTemporaryFile(
        suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=False
    )
    tmp_copy.write(output_content)
    return tmp_copy

In [12]:
convert_data = Dataset(make_tmp_copy(convert_output).name)
convert_meatadata = {
    "year": convert_data.history.split()[5],
    "month": convert_data.history.split()[2],
    "day": convert_data.history.split()[3],
    "source": convert_data.source,
    "RvicPourPointsFile": convert_data.RvicPourPointsFile,
    "RvicUHFile": convert_data.RvicUHFile,
    "RvicFdrFile": convert_data.RvicFdrFile,
    "RvicDomainFile": convert_data.RvicDomainFile,
}

In [13]:
convert_config = read_config("tests/data/configs/convert_mixed.cfg")
expected_metadata = {
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "source": "/home/sangwonl/Desktop/osprey/venv/bin/osprey",
    "RvicPourPointsFile": convert_config["UHS_FILES"]["STATION_FILE"].split("/")[-1],
    "RvicUHFile": "unknown",
    "RvicFdrFile": "unknown",
    "RvicDomainFile": convert_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

assert convert_meatadata == expected_metadata

In [14]:
metadata = Dataset(make_tmp_copy(params_output).name)
metadata

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: RVIC parameter file
    comment: Output from the RVIC Streamflow Routing Model.
    Conventions: CF-1.6
    history: Created: Mon Sep 21 15:54:29 2020
    source: /home/sangwonl/Desktop/osprey/venv/bin/osprey
    institution: University of Washington
    hostname: devel2
    username: sangwonl
    references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
    version: 1.1.2
    RvicPourPointsFile: sample_pour.txt
    RvicUHFile: uhbox.csv
    RvicFdrFile: sample_flow_parameters.nc
    RvicDomainFile: sample_routing_domain.nc
    featureType: timeSeries
    dimensions(sizes): timesteps(100), outlets(1), nc_chars(256), sources(1), tracers(1)
    variables(dimensions): float64 timesteps(timesteps), int32 full_time_length(), int32 subset_length(), float64 unit_hydrograph_dt(), float64 outlet_lon(outlets), float64 outlet_lat(outlets), int32 outlet_x_ind(outlets), int

In [15]:
metadata = Dataset(make_tmp_copy(convolve_output).name)
metadata

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: RVIC history file
    comment: Output from the RVIC Streamflow Routing Model.
    Conventions: CF-1.6
    history: Created: Mon Sep 21 15:54:55 2020
    source: /home/sangwonl/Desktop/osprey/venv/bin/osprey
    institution: University of Washington
    hostname: devel2
    username: sangwonl
    casename: sample
    casestr: historical
    references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
    version: 1.1.2
    RvicPourPointsFile: sample_pour.txt
    RvicUHFile: uhbox.csv
    RvicFdrFile: sample_flow_parameters.nc
    RvicDomainFile: sample_routing_domain.nc
    featureType: timeSeries
    dimensions(sizes): time(31), nv(2), outlets(1), nc_chars(64)
    variables(dimensions): float32 time(time), float32 time_bnds(time, nv), float32 lon(outlets), float32 lat(outlets), int32 outlet_x_ind(outlets), int32 outlet_y_ind(outlets), int32 outlet_decomp_ind(outle

In [16]:
metadata = Dataset(make_tmp_copy(full_rvic_output).name)
metadata

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: RVIC history file
    comment: Output from the RVIC Streamflow Routing Model.
    Conventions: CF-1.6
    history: Created: Mon Sep 21 15:55:25 2020
    source: /home/sangwonl/Desktop/osprey/venv/bin/osprey
    institution: University of Washington
    hostname: devel2
    username: sangwonl
    casename: sample
    casestr: historical
    references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
    version: 1.1.2
    RvicPourPointsFile: sample_pour.txt
    RvicUHFile: uhbox.csv
    RvicFdrFile: sample_flow_parameters.nc
    RvicDomainFile: sample_routing_domain.nc
    featureType: timeSeries
    dimensions(sizes): time(31), nv(2), outlets(1), nc_chars(256)
    variables(dimensions): float32 time(time), float32 time_bnds(time, nv), float32 lon(outlets), float32 lat(outlets), int32 outlet_x_ind(outlets), int32 outlet_y_ind(outlets), int32 outlet_decomp_ind(outl